In [ ]:
#Data Wraggling. Collection and clean up of the data.
#Taking 15 years of electrical outage data to predict future outages
#Cleaning up the data so we can get the duration of the outage along with all of the dates and times standardized.
#Handling missing data


In [1]:
import pandas as pd
import numpy as np
import re

outage_df=pd.read_csv('C:/Users/hmkfs/Desktop/Springboard/Capstone 3/Data file/outagedata.csv')

In [2]:
outage_df.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags
0,Severe Weather - Thunderstorms,2014,6/30/2014,8:00 PM,7/2/2014,6:30 PM,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm"
1,Severe Weather - Thunderstorms,2014,6/30/2014,11:20 PM,7/1/2014,5:00 PM,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm"
2,Severe Weather - Thunderstorms,2014,6/30/2014,5:55 PM,7/1/2014,2:53 AM,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm"
3,Fuel Supply Emergency - Coal,2014,6/27/2014,1:21 PM,Unknown,Unknown,We Energies,Wisconsin,MRO,Unknown,Unknown,"fuel supply emergency, coal"
4,Physical Attack - Vandalism,2014,6/24/2014,2:54 PM,6/24/2014,2:55 PM,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical"


In [3]:
outage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1652 entries, 0 to 1651
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Event Description             1652 non-null   object
 1   Year                          1652 non-null   int64 
 2   Date Event Began              1652 non-null   object
 3   Time Event Began              1643 non-null   object
 4   Date of Restoration           1638 non-null   object
 5   Time of Restoration           1632 non-null   object
 6   Respondent                    1652 non-null   object
 7   Geographic Areas              1651 non-null   object
 8   NERC Region                   1650 non-null   object
 9   Demand Loss (MW)              1246 non-null   object
 10  Number of Customers Affected  1434 non-null   object
 11  Tags                          1651 non-null   object
dtypes: int64(1), object(11)
memory usage: 155.0+ KB


In [4]:
outage_df.describe()

,Year
count,1652.000000
mean,2009.224576
std,3.544618
min,2000.000000
25%,2007.000000
50%,2010.000000
75%,2012.000000
max,2014.000000


In [5]:
outage_df.count()

Event Description               1652
Year                            1652
Date Event Began                1652
Time Event Began                1643
Date of Restoration             1638
Time of Restoration             1632
Respondent                      1652
Geographic Areas                1651
NERC Region                     1650
Demand Loss (MW)                1246
Number of Customers Affected    1434
Tags                            1651
dtype: int64

In [6]:
#check for missing values
print(outage_df.isna().sum())

Event Description                 0
Year                              0
Date Event Began                  0
Time Event Began                  9
Date of Restoration              14
Time of Restoration              20
Respondent                        0
Geographic Areas                  1
NERC Region                       2
Demand Loss (MW)                406
Number of Customers Affected    218
Tags                              1
dtype: int64


In [7]:
# Handle missing values
outage_df['Date of Restoration'] = outage_df['Date of Restoration'].replace('Unknown', np.nan)
outage_df = outage_df.dropna(subset=['Date of Restoration'])
outage_df['Time of Restoration'] = outage_df['Time of Restoration'].fillna('12:00 PM')

In [8]:
# Function to standardize time format and handle some NaN
def standardize_time_format(time_str):
    if pd.isna(time_str):
        return time_str
    # Remove seconds and handle cases like 'p.m.' and 'a.m.'
    time_str = re.sub(r'(\d{1,2}:\d{2})\s*(a\.?m\.?|p\.?m\.?)', r'\1 \2', time_str, flags=re.IGNORECASE)
    time_str = time_str.strip().replace('.', '')  # Remove any remaining periods
    return time_str

In [9]:
# Apply standardization
outage_df['Time Event Began'] = outage_df['Time Event Began'].astype(str).apply(standardize_time_format)
outage_df['Time of Restoration'] = outage_df['Time of Restoration'].astype(str).apply(standardize_time_format)

In [10]:
# Convert time strings to datetime objects
outage_df['Time Event Began'] = pd.to_datetime(outage_df['Time Event Began'], format='%I:%M %p', errors='coerce').dt.time
outage_df['Time of Restoration'] = pd.to_datetime(outage_df['Time of Restoration'], format='%I:%M %p', errors='coerce').dt.time

In [11]:
# Clean Date columns and handle invalid data
def clean_date(date_str):
    # Convert to string to handle different types
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str)
    if date_str.lower() == 'unknown':
        return pd.NaT
    return date_str


In [12]:
# Apply cleaning to date columns
outage_df['Date Event Began'] = outage_df['Date Event Began'].astype(str).apply(clean_date)
outage_df['Date of Restoration'] = outage_df['Date of Restoration'].astype(str).apply(clean_date)


In [13]:
# Convert date strings to datetime objects with error handling
outage_df['Date Event Began'] = pd.to_datetime(outage_df['Date Event Began'], format='%m/%d/%Y', errors='coerce')
outage_df['Date of Restoration'] = pd.to_datetime(outage_df['Date of Restoration'], format='%m/%d/%Y', errors='coerce')


In [14]:
date_format = '%m/%d/%Y %I:%M %p'

In [15]:
# Combine date and time to create datetime columns
outage_df['Event Start'] = pd.to_datetime(outage_df['Date Event Began'].astype(str) + ' ' + outage_df['Time Event Began'].astype(str), errors='coerce')
outage_df['Event End'] = pd.to_datetime(outage_df['Date of Restoration'].astype(str) + ' ' + outage_df['Time of Restoration'].astype(str), errors='coerce')


In [16]:
# Calculate duration of the outage
outage_df['Duration'] = outage_df['Event End'] - outage_df['Event Start']

In [17]:
# Display DataFrame to check results
outage_df.head()

,Event Description,Year,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Respondent,Geographic Areas,NERC Region,Demand Loss (MW),Number of Customers Affected,Tags,Event Start,Event End,Duration
0,Severe Weather - Thunderstorms,2014,2014-06-30,20:00:00,2014-07-02,18:30:00,Exelon Corporation/ComEd,Illinois,RFC,Unknown,"420,000","severe weather, thunderstorm",2014-06-30 20:00:00,2014-07-02 18:30:00,1 days 22:30:00
1,Severe Weather - Thunderstorms,2014,2014-06-30,23:20:00,2014-07-01,17:00:00,Northern Indiana Public Service Company,North Central Indiana,RFC,Unknown,"127,000","severe weather, thunderstorm",2014-06-30 23:20:00,2014-07-01 17:00:00,0 days 17:40:00
2,Severe Weather - Thunderstorms,2014,2014-06-30,17:55:00,2014-07-01,02:53:00,We Energies,Southeast Wisconsin,MRO,424,"120,000","severe weather, thunderstorm",2014-06-30 17:55:00,2014-07-01 02:53:00,0 days 08:58:00
4,Physical Attack - Vandalism,2014,2014-06-24,14:54:00,2014-06-24,14:55:00,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical",2014-06-24 14:54:00,2014-06-24 14:55:00,0 days 00:01:00
5,Physical Attack - Vandalism,2014,2014-06-19,08:47:00,2014-06-19,08:48:00,Tennessee Valley Authority,"Nashville, Tennessee",SERC,Unknown,Unknown,"vandalism, physical",2014-06-19 08:47:00,2014-06-19 08:48:00,0 days 00:01:00


In [18]:
# Save the cleaned dataframe to a new CSV file
outage_df.to_csv('cleaned_outagedata.csv', index=False)